Links:
* [h3 edges](https://h3geo.org/docs/highlights/aggregation)


## Initial setup

In [ ]:
from shapely.geometry import Point, Polygon, GeometryCollection, shape
from datetime import datetime
import h3.api.numpy_int as h3
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
import branca
import geojson
from route3_road_client import Server, make_graph_handle

server = Server(hostport="127.0.0.1:7088")
print(server.version())

graph_info = server.graph_info()
for g in graph_info.graphs:
    print(g)
    
graph_handle = make_graph_handle("germany", 10)

#graph_h3_resolution = graph_info.h3_resolution

def dataframe_to_geodataframe(df: pd.DataFrame, column_name: str = "h3index") -> gpd.GeoDataFrame:
    return gpd.GeoDataFrame(df,
                            geometry=[Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in
                                      np.nditer(df[column_name].to_numpy())],
                            crs=4326)


# test POIs - fast food shops. Can be created using the top-level justfile
poi_df = gpd.read_file("../../data/hospital.geojson")
del poi_df['@id']
print(f"Using {len(poi_df)} POI locations")

# only use points
poi_df["geometry"] = poi_df.geometry.centroid

# create the h3indexes for the geometries to have them available for the
# attribution in maps later
poi_df["h3index"] = poi_df.geometry.map(lambda geom: h3.geo_to_h3(geom.y, geom.x, graph_handle.h3_resolution))

poi_df

## Create a disturbance which hinders traffic

You can use [geojson.io](http://geojson.io) to draw a featurecollection and paste it into the string bellow.

In [ ]:
disturbance_geojson_string = """
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              10.87320327758789,
              47.91300548190604
            ],
            [
              10.876636505126953,
              47.91300548190604
            ],
            [
              10.876636505126953,
              47.91611193960253
            ],
            [
              10.87320327758789,
              47.91611193960253
            ],
            [
              10.87320327758789,
              47.91300548190604
            ]
          ]
        ]
      }
    }
  ]
}
"""


disturbance = GeometryCollection([shape(feature["geometry"]).buffer(0) for feature in geojson.loads(disturbance_geojson_string)["features"]])
disturbance

## Route and Analyze

In [ ]:
t_start = datetime.now()
object_id, dsp_df = server.differential_shortest_path(
    graph_handle,
    disturbance, 
    25000.0, 
    poi_df.geometry, 
    num_destinations_to_reach=2,
    num_gap_cells_to_graph=3,
    downsampled_prerouting=True
)
print(f"took {datetime.now() - t_start}")
print(f"id: {object_id}")

geo_df = dataframe_to_geodataframe(dsp_df, column_name="h3index_origin")
geo_df["cost_increase"] = geo_df["avg_cost_with_disturbance"] - geo_df["avg_cost_without_disturbance"]
geo_df

... now map it ...

In [ ]:
map_content_disolved = geo_df.dissolve().geometry[0].convex_hull
map_center = map_content_disolved.centroid

m = folium.Map(location=[map_center.y, map_center.x], zoom_start=11, tiles='OpenStreetMap', 
               #crs="EPSG4326"
              )

poi_df_subset = poi_df.loc[poi_df.geometry.within(map_content_disolved.buffer(0.8))]

folium.GeoJson(
    # use only a subset of the POIs to speed up rendering
    poi_df_subset,
    style_function=lambda x: {
        'fillColor': "#c00", 
        'color' : '#800',
        'weight' : 1,
        'fillOpacity' : 0.3,
    },
    tooltip=folium.features.GeoJsonTooltip([
        "name",
        "h3index",
    ])
    ).add_to(m) 

folium.GeoJson(
    disturbance,
    style_function=lambda x: {
        'fillColor': "#c00", 
        'color' : '#800',
        'weight' : 1,
        'fillOpacity' : 0.3,
    },
    ).add_to(m)    
    

colorscale_cell_worsen = branca.colormap.LinearColormap(
    ((1.,1.,1.), (0.0,0.0,1.0)), 
    vmin=0.0, 
    vmax=geo_df['cost_increase'].max()
) 
   
def style_cell(feature):
    cost_inc = feature['properties']['cost_increase'] 
    if cost_inc is None:
        fill = '#f00' # no route found
    else: 
        fill = colorscale_cell_worsen(cost_inc)
    return {
        'fillColor': fill, 
        'color' : '#555',
        'weight' : 1,
        'fillOpacity' : 0.8,
    }

folium.GeoJson(
    geo_df,
    style_function=style_cell,
    tooltip=folium.features.GeoJsonTooltip([
        "h3index_origin",
        "avg_cost_without_disturbance",
        "avg_cost_with_disturbance",
        "cost_increase",
        "population_origin", 
        "preferred_dest_h3index_without_disturbance", 
        "preferred_dest_h3index_with_disturbance",
        "num_reached_without_disturbance",
        "num_reached_with_disturbance",
    ])
    ).add_to(m)    
    
m

In [ ]:
some_cells = list(geo_df.sort_values(["cost_increase"], ascending=False).h3index_origin)[:2]

route_gdf = server.get_differential_shortest_path_routes(object_id, some_cells)
len(route_gdf)
route_gdf

In [ ]:
def style_route(feature):
    if feature['properties']['with_disturbance']:
        color = '#f00'
    else: 
        color = "#0f0"
    return { 
        'color' : color,
        'weight' : 2,
    }

folium.GeoJson(
    route_gdf, #[route_gdf['with_disturbance'] == 0],
    style_function=style_route,
    tooltip=folium.features.GeoJsonTooltip([
        "h3index_origin",
        "h3index_destination",
        "cost",
    ])
    ).add_to(m)    
    
m

## Retrieve results again using the ID

In [ ]:
object_id, res_df = server.get_differential_shortest_path(object_id)
res_df